# Análise de Resultados - Motor de Correspondência

Este notebook realiza análises detalhadas dos resultados do motor de correspondência.

In [ ]:
# Importar configurações
%run ./00_configuracao_inicial.ipynb

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

print("="*60)
print("ANÁLISE FINAL - MOTOR DE CORRESPONDÊNCIA")
print("="*60)

In [ ]:
# Carregar resultados finais
try:
    df_resultado_final = read_delta_table(f"{PATH_GOLD}/matches_validados")
    print(f"✓ Resultados carregados: {df_resultado_final.count()} registros")
except Exception as e:
    print(f"✗ Erro ao carregar resultados: {e}")
    print("Tentando carregar matches sem validação...")
    df_resultado_final = read_delta_table(PATH_MATCHES)

In [ ]:
# Estatísticas gerais
print("\n" + "="*60)
print("ESTATÍSTICAS GERAIS")
print("="*60)

estatisticas_gerais = df_resultado_final.agg(
    count("*").alias("total_matches"),
    count(when(col("score_final_validado") >= 0.9, 1)).alias("matches_alta_confianca"),
    count(when((col("score_final_validado") >= 0.7) & (col("score_final_validado") < 0.9), 1)).alias("matches_media_confianca"),
    count(when(col("score_final_validado") < 0.7, 1)).alias("matches_baixa_confianca"),
    avg("score_final_validado").alias("score_medio"),
    min("score_final_validado").alias("score_minimo"),
    max("score_final_validado").alias("score_maximo")
)

estatisticas_gerais.show()

In [ ]:
# Análise por cidade
print("\n" + "="*60)
print("ANÁLISE POR CIDADE")
print("="*60)

coluna_cidade = "cidade_entrada" if "cidade_entrada" in df_resultado_final.columns else "cidade"
coluna_uf = "uf_entrada" if "uf_entrada" in df_resultado_final.columns else "uf"

analise_cidade = df_resultado_final.groupBy(coluna_cidade, coluna_uf).agg(
    count("*").alias("total_matches"),
    avg("score_final_validado").alias("score_medio"),
    count(when(col("score_final_validado") >= 0.9, 1)).alias("alta_confianca"),
    count(when((col("score_final_validado") >= 0.7) & (col("score_final_validado") < 0.9), 1)).alias("media_confianca")
).orderBy(col("total_matches").desc())

analise_cidade.show()

In [ ]:
# Top 50 matches com maior score
print("\n" + "="*60)
print("TOP 50 MATCHES - MAIOR SCORE")
print("="*60)

colunas_select = [
    "id_entrada",
    "endereco_livre",
    "nome_logradouro_ouro",
    "numero_ouro",
    "bairro_ouro",
    "cidade_ouro",
    "score_final_validado"
]

# Adicionar colunas opcionais se existirem
colunas_opcionais = ["distancia_metros", "validacao_geografica"]
for coluna in colunas_opcionais:
    if coluna in df_resultado_final.columns:
        colunas_select.append(coluna)

top_matches = df_resultado_final.select(*colunas_select).orderBy(
    col("score_final_validado").desc()
).limit(50)

top_matches.show(50, truncate=False)

In [ ]:
# Casos sem match
print("\n" + "="*60)
print("ENDEREÇOS SEM MATCH")
print("="*60)

# Carregar endereços de entrada
df_entrada = read_delta_table(PATH_ENDERECOS_NORMALIZADOS)

# Encontrar endereços sem match
ids_com_match = df_resultado_final.select("id_entrada").distinct()
df_sem_match = df_entrada.join(
    ids_com_match,
    df_entrada.id == ids_com_match.id_entrada,
    "left_anti"
)

total_sem_match = df_sem_match.count()
total_entrada = df_entrada.count()
taxa_match = ((total_entrada - total_sem_match) / total_entrada * 100) if total_entrada > 0 else 0

print(f"Total de endereços de entrada: {total_entrada}")
print(f"Endereços com match: {total_entrada - total_sem_match}")
print(f"Endereços sem match: {total_sem_match}")
print(f"Taxa de match: {taxa_match:.2f}%")

if total_sem_match > 0:
    print("\nExemplos de endereços sem match:")
    df_sem_match.select("id", "endereco_livre", "cidade", "uf").show(20, truncate=False)

In [ ]:
# Distribuição de scores
print("\n" + "="*60)
print("DISTRIBUIÇÃO DE SCORES")
print("="*60)

df_distribuicao = df_resultado_final.select(
    when(col("score_final_validado") >= 0.9, "0.90-1.00")
    .when(col("score_final_validado") >= 0.8, "0.80-0.89")
    .when(col("score_final_validado") >= 0.7, "0.70-0.79")
    .when(col("score_final_validado") >= 0.6, "0.60-0.69")
    .otherwise("0.00-0.59")
    .alias("faixa_score"),
    col("score_final_validado")
).groupBy("faixa_score").agg(
    count("*").alias("quantidade"),
    avg("score_final_validado").alias("score_medio")
).orderBy("faixa_score")

df_distribuicao.show()

In [ ]:
# Análise de validação geográfica (se disponível)
if "validacao_geografica" in df_resultado_final.columns:
    print("\n" + "="*60)
    print("ANÁLISE DE VALIDAÇÃO GEOGRÁFICA")
    print("="*60)
    
    estatisticas_validacao = df_resultado_final.agg(
        count("*").alias("total_matches"),
        count(when(col("validacao_geografica") == "ALTA", 1)).alias("validacao_alta"),
        count(when(col("validacao_geografica") == "MEDIA", 1)).alias("validacao_media"),
        count(when(col("validacao_geografica") == "BAIXA", 1)).alias("validacao_baixa"),
        avg("distancia_metros").alias("distancia_media_metros")
    )
    
    estatisticas_validacao.show()
    
    # Distribuição de distâncias
    if "distancia_metros" in df_resultado_final.columns:
        print("\nDistribuição de distâncias:")
        df_distancias = df_resultado_final.select(
            when(col("distancia_metros") <= 10, "0-10m")
            .when(col("distancia_metros") <= 50, "11-50m")
            .when(col("distancia_metros") <= 100, "51-100m")
            .when(col("distancia_metros") <= 500, "101-500m")
            .otherwise(">500m")
            .alias("faixa_distancia"),
            col("distancia_metros")
        ).groupBy("faixa_distancia").agg(
            count("*").alias("quantidade")
        ).orderBy("faixa_distancia")
        
        df_distancias.show()

In [ ]:
# Resumo final
print("\n" + "="*60)
print("RESUMO FINAL")
print("="*60)

total_entrada = df_entrada.count()
total_matches = df_resultado_final.count()
total_sem_match = total_entrada - df_resultado_final.select("id_entrada").distinct().count()

print(f"Total de endereços processados: {total_entrada}")
print(f"Total de matches encontrados: {total_matches}")
print(f"Endereços sem match: {total_sem_match}")
print(f"Taxa de match: {((total_entrada - total_sem_match) / total_entrada * 100):.2f}%")

score_medio = df_resultado_final.agg(avg("score_final_validado")).collect()[0][0]
print(f"Score médio: {score_medio:.4f}")

print("\n✓ Análise concluída!")